<a href="https://colab.research.google.com/github/hkhong72/big_data/blob/main/221101_7_Chap19_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras import models, layers
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [ ]:
# 생성자 모델 만들기
generator = models.Sequential()
generator.add(layers.Dense(128*7*7, input_dim=100, activation=layers.LeakyReLU(0.2)))
generator.add(layers.BatchNormalization()) # 스케일자체를 축소(0~1사이로)
generator.add(layers.Reshape((7, 7, 128))) # 7by7이미지가 128개채널 -> 3차원으로변경
generator.add(layers.UpSampling2D()) 
# MaxPooling에 반대. width, heights 두배씩, 총면적 -> 4배
generator.add(layers.Conv2D(64, kernel_size=5, padding='same')) 
#입력이미지와 출력이미지에 사이즈변화X, 2by2씩 붙여야함
generator.add(layers.BatchNormalization()) 
generator.add(layers.Activation(layers.LeakyReLU(0.2))) 
generator.add(layers.UpSampling2D())
generator.add(layers.Conv2D(1, kernel_size=5, padding='same', activation='tanh'))
generator.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 6272)              633472    
                                                                 
 batch_normalization_3 (Batc  (None, 6272)             25088     
 hNormalization)                                                 
                                                                 
 reshape_1 (Reshape)         (None, 7, 7, 128)         0         
                                                                 
 up_sampling2d_2 (UpSampling  (None, 14, 14, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        204864    
                                                                 
 batch_normalization_4 (Batc  (None, 14, 14, 64)      

In [ ]:
# 1. 입력값100 * 128*7*7 + 128*7*7 = 633472
# 2. 입력값128*7*7(6272) * 3 + 6272 = 25088
# 3. 128 * w(5, 5) * 64(커널갯수) + b64 = 204864
# 4. 64 * w192 + 64 = 256
# 5. (64 + (28-3)) * 1 + 1 = 1601

In [ ]:
# 판별자 모델 만들기
discriminator = models.Sequential()
discriminator.add(layers.Conv2D(64, kernel_size=5, strides=2, input_shape=(28,28,1), padding="same"))
discriminator.add(layers.Activation(layers.LeakyReLU(0.2)))
discriminator.add(layers.Dropout(0.3))
discriminator.add(layers.Conv2D(128, kernel_size=5, strides=2, padding="same"))
discriminator.add(layers.Activation(layers.LeakyReLU(0.2)))
discriminator.add(layers.Dropout(0.3))
discriminator.add(layers.Flatten())
discriminator.add(layers.Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable=False # 별도학습을 시키기위해

In [ ]:
ginput = layers.Input(shape=(100,)) # 생성자는 처음에100개노이즈들어옴
dis_output = discriminator(generator(ginput))
gan = models.Model(ginput, dis_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')
gan.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential_2 (Sequential)   (None, 28, 28, 1)         865281    
                                                                 
 sequential_6 (Sequential)   (None, 1)                 212865    
                                                                 
Total params: 1,078,146
Trainable params: 852,609
Non-trainable params: 225,537
_________________________________________________________________


In [ ]:
# 신경망을 실행시키는 함수 만들기
def gan_train(epoch, batch_size, saving_interval):
    (X_train, _), (_, _) = mnist.load_data()
    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype(float)
    X_train = (X_train - 127.5) / 127.5
    
    true = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    for i in range(epoch):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        d_loss_real = discriminator.train_on_batch(imgs, true)
        
        noise = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(noise)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        g_loss = gan.train_on_batch(noise, true)
        
        print('epoch:%d' % i, 'd_loss:%.4f' % d_loss, 'g_loss:%.4f' % g_loss)
    
        if i % saving_interval == 0:
            
            #r, c = 5, 5
            noise = np.random.normal(0, 1, (25, 100))
            gen_imgs = generator.predict(noise)

            # Rescale images 0 - 1
            gen_imgs = 0.5 * gen_imgs + 0.5

            fig, axs = plt.subplots(5, 5)
            count = 0
            for j in range(5):
                 for k in range(5):
                    axs[j, k].imshow(gen_imgs[count, :, :, 0], cmap='gray')
                    axs[j, k].axis('off')
                    count += 1
            fig.savefig("./gan_images/gan_mnist_%d.png" % i)

In [ ]:
gan_train(2001,32,200)

epoch:0 d_loss:0.8777 g_loss:2.4526
epoch:1 d_loss:0.6856 g_loss:3.0332
epoch:2 d_loss:0.7144 g_loss:3.1283
epoch:3 d_loss:0.6410 g_loss:2.7500
epoch:4 d_loss:0.4442 g_loss:2.8152
epoch:5 d_loss:0.4054 g_loss:2.3460
epoch:6 d_loss:0.4486 g_loss:2.3793
epoch:7 d_loss:0.3540 g_loss:1.7527
epoch:8 d_loss:0.4789 g_loss:1.7556
epoch:9 d_loss:0.7795 g_loss:1.9602
epoch:10 d_loss:0.3470 g_loss:2.7152
epoch:11 d_loss:0.2629 g_loss:3.6287
epoch:12 d_loss:0.3710 g_loss:3.8117
epoch:13 d_loss:0.2748 g_loss:3.7670
epoch:14 d_loss:0.2345 g_loss:3.2116
epoch:15 d_loss:0.4731 g_loss:2.4855
epoch:16 d_loss:0.3792 g_loss:2.5277
epoch:17 d_loss:0.2103 g_loss:2.7807
epoch:18 d_loss:0.2384 g_loss:2.8623
epoch:19 d_loss:0.2235 g_loss:2.9596
epoch:20 d_loss:0.2948 g_loss:2.9999
epoch:21 d_loss:0.2150 g_loss:3.4464
epoch:22 d_loss:0.3159 g_loss:2.4898
epoch:23 d_loss:0.5103 g_loss:2.0872
epoch:24 d_loss:0.4254 g_loss:1.4843
epoch:25 d_loss:0.4981 g_loss:1.6996
epoch:26 d_loss:0.5230 g_loss:1.7642
epoch:27 d_